# Using ResNet-50 to encode images

**In this section:**
- Use ResNet-50 to encode images
- Replace normal convolutional layers with atrous convolutional layers
- Replace atrous convolutional layers with depthwise convolutional layers

In [25]:
import numpy as np
from utils import *
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D, SeparableConv2D, Concatenate
from keras.models import Model, load_model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
import scipy.misc
from matplotlib.pyplot import imshow
%matplotlib inline


In [39]:
#Import the Cifar10-dataset
X,Y,_ = load_cifar_10_data()
X_train = X[0:10000]
Y_train = Y[:,0:10000]

In [40]:
print(X_train.shape)
print(Y_train.shape)

(10000, 32, 32, 3)
(10, 10000)


In [26]:
def atrous_convolution_block(X,f):
    """
    Applies atrous convolution with different rates and combines them to form feature map
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    
    
    Returns:
    X -- output of atrous convolution block of tensor shape (m, n_H, n_W, n_C)
    """
    
    # Retrieve all the filters for the three atrous convolutions
    F1, F2 ,F3 = f
    
    #Apply three atrous convolutions in parallel with same padding to remain output size
    A1 = SeparableConv2D(filters = F1, kernel_size = [3,3], strides = [1,1], dilation_rate = [2,2],padding = "same",kernel_initializer = glorot_uniform())(X)
    A2 = SeparableConv2D(filters = F2, kernel_size = [3,3], strides = [1,1], dilation_rate = [3,3],padding = "same",kernel_initializer = glorot_uniform())(X)
    A3 = SeparableConv2D(filters = F3, kernel_size = [3,3], strides = [1,1], dilation_rate = [4,4],padding = "same",kernel_initializer = glorot_uniform())(X)

    #Concatenate the three outputs A1, A2 and A3
    X = Concatenate(axis = -1)([A1,A2,A3])
    
    return X
    
    

In [27]:
def identity_block(X, f, filters, atrous_filters, stage, block):
    """
    Implementation of the identity block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    
    Returns:
    X -- output of the identity block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    # Save the input value. You'll need this later to add back to the main path. 
    X_shortcut = X
    
    # First component of main path
    X = Conv2D(filters = F1, kernel_size = (1, 1), strides = (1,1), padding = 'valid', name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###
    
    # Second component of main path (≈3 lines)
    X = atrous_convolution_block(X,atrous_filters)
    X = SeparableConv2D(filters = F2, kernel_size = (f,f), dilation_rate = (2,2), strides = (1,1), padding="same",name = conv_name_base + '2b', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), padding="valid",name = conv_name_base + '2c', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2c')(X)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation("relu")(X)
    
    ### END CODE HERE ###
    
    return X

In [28]:
def convolutional_block(X, f, filters, stage, block, s = 2):
    """
    Implementation of the convolutional block as defined in Figure 4
    
    Arguments:
    X -- input tensor of shape (m, n_H_prev, n_W_prev, n_C_prev)
    f -- integer, specifying the shape of the middle CONV's window for the main path
    filters -- python list of integers, defining the number of filters in the CONV layers of the main path
    stage -- integer, used to name the layers, depending on their position in the network
    block -- string/character, used to name the layers, depending on their position in the network
    s -- Integer, specifying the stride to be used
    
    Returns:
    X -- output of the convolutional block, tensor of shape (n_H, n_W, n_C)
    """
    
    # defining name basis
    conv_name_base = 'res' + str(stage) + block + '_branch'
    bn_name_base = 'bn' + str(stage) + block + '_branch'
    
    # Retrieve Filters
    F1, F2, F3 = filters
    
    # Save the input value
    X_shortcut = X


    ##### MAIN PATH #####
    # First component of main path 
    X = Conv2D(F1, (1, 1), strides = (s,s), name = conv_name_base + '2a', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2a')(X)
    X = Activation('relu')(X)
    
    ### START CODE HERE ###

    # Second component of main path (≈3 lines)
    X = SeparableConv2D(filters = F2, kernel_size = (f,f), strides = (1,1),padding="same",name = conv_name_base + '2b',kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = bn_name_base + '2b')(X)
    X = Activation('relu')(X)

    # Third component of main path (≈2 lines)
    X = Conv2D(filters = F3, kernel_size = (1,1), strides = (1,1), dilation_rate = (2,2),padding="valid",name = conv_name_base + '2c',kernel_initializer = glorot_uniform(seed = 0))(X)
    X = BatchNormalization(axis = 3, name=bn_name_base + '2c')(X)

    ##### SHORTCUT PATH #### (≈2 lines)
    X_shortcut = Conv2D(filters = F3,kernel_size = (1,1),strides = (s,s),padding = "valid", name=conv_name_base + '1',kernel_initializer = glorot_uniform(seed = 0))(X_shortcut)
    X_shortcut = BatchNormalization(axis = 3, name=bn_name_base + '1')(X_shortcut)

    # Final step: Add shortcut value to main path, and pass it through a RELU activation (≈2 lines)
    X = Add()([X,X_shortcut])
    X = Activation('relu')(X)
    
    ### END CODE HERE ###
    
    return X

In [29]:
def ResNet50(input_shape = (64, 64, 3), classes = 6):
    """
    Implementation of the popular ResNet50 the following architecture:
    CONV2D -> BATCHNORM -> RELU -> MAXPOOL -> CONVBLOCK -> IDBLOCK*2 -> CONVBLOCK -> IDBLOCK*3
    -> CONVBLOCK -> IDBLOCK*5 -> CONVBLOCK -> IDBLOCK*2 -> AVGPOOL -> TOPLAYER

    Arguments:
    input_shape -- shape of the images of the dataset
    classes -- integer, number of classes

    Returns:
    model -- a Model() instance in Keras
    """
    
    # Define the input as a tensor with shape input_shape
    X_input = Input(input_shape)

    
    # Zero-Padding
    X = ZeroPadding2D((3, 3))(X_input)
    
    # Stage 1
    X = Conv2D(64, (7, 7), strides = (2, 2), name = 'conv1', kernel_initializer = glorot_uniform(seed=0))(X)
    X = BatchNormalization(axis = 3, name = 'bn_conv1')(X)
    X = Activation('relu')(X)
    X = MaxPooling2D((3, 3), strides=(2, 2))(X)

    # Stage 2
    X = convolutional_block(X, f = 3, filters = [64, 64, 256], stage = 2, block='a', s = 1)
    X = identity_block(X, 3, [64, 64, 256], [64,64,256], stage=2, block='b')
    X = identity_block(X, 3, [64, 64, 256], [64,64,256], stage=2, block='c')

    ### START CODE HERE ###

    # Stage 3 (≈4 lines)
    X = convolutional_block(X, f = 3, filters = [128,128,512], stage = 3, block='a',s = 2 )
    X = identity_block(X, f = 3, filters = [128,128,512], atrous_filters = [128,128,512], stage = 3, block = 'b')
    X = identity_block(X, f = 3, filters = [128,128,512], atrous_filters = [128,128,512], stage = 3, block = 'c')
    X = identity_block(X, f = 3, filters = [128,128,512], atrous_filters = [128,128,512], stage = 3, block = 'd')

    # Stage 4 (≈6 lines)
    X = convolutional_block(X, f = 3, filters = [256,256,1024], stage = 4, block='a', s = 2)
    X = identity_block(X, f = 3, filters = [256,256,1024], atrous_filters = [256,256,1024], stage = 4, block = 'b')
    X = identity_block(X, f = 3, filters = [256,256,1024], atrous_filters = [256,256,1024], stage = 4, block = 'c')
    X = identity_block(X, f = 3, filters = [256,256,1024], atrous_filters = [256,256,1024], stage = 4, block = 'd')
    X = identity_block(X, f = 3, filters = [256,256,1024], atrous_filters = [256,256,1024], stage = 4, block = 'e')
    X = identity_block(X, f = 3, filters = [256,256,1024], atrous_filters = [256,256,1024], stage = 4, block = 'f')


    # MAXPOOL (≈1 line). Use "X = AveragePooling2D(...)(X)"
    X = AveragePooling2D()(X)
    
    ### END CODE HERE ###

    # output layer
    X = Flatten()(X)
    X = Dense(classes, activation='softmax', name='fc' + str(classes), kernel_initializer = glorot_uniform())(X)
    
    
    # Create model
    model = Model(inputs = X_input, outputs = X, name='ResNet50')

    return model

In [ ]:
model = ResNet50(input_shape = (32, 32, 3), classes = 10)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train,Y_train.T,epochs = 2,batch_size = 1000)


Epoch 1/2
 1000/10000 [==>...........................] - ETA: 46:54 - loss: 3.0130 - accuracy: 0.1260

In [31]:
model.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
zero_padding2d_4 (ZeroPadding2D (None, 38, 38, 3)    0           input_4[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 16, 16, 64)   9472        zero_padding2d_4[0][0]           
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 16, 16, 64)   256         conv1[0][0]                      
___________________________________________________________________________________________

In [35]:
print(Y_train.shape)

(10, 50000)
